In [1]:
# expand cells to the 95% of the display width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 95% !important; }</style>"))

# Automatic rule-based modeling of the *Escherichia coli* Central Carbon metabolism

## Preamble: load *Atlas*

In [2]:
# testing source code
import sys
sys.path.append("../..") # If installed from GitHub and this notebooks is executed from the example/subfolder directory.

import atlas_rbm.atlas as atlas
import atlas_rbm.utils as utils
import atlas_rbm.export as export
import atlas_rbm.simulation as simulation

## Modeling metabolism

In [3]:
utils.execPToolsDocker('ptools-v22')

Doing nothing since PathwayTools is running.


In [4]:
genes = ['aceA', 'aceB', 'aceE', 'aceF', 'acnA', 'acnB', 'add', 'adk', 'amn', 'ansA', 
    'ansB', 'aphA', 'apt', 'argD', 'aroA', 'aroB', 'aroC', 'aroD', 'aroE', 'aroF', 
    'aroG', 'aroH', 'aroK', 'aroL', 'asd', 'asnA', 'asnB', 'aspA', 'aspC', 'carA', 
    'carB', 'cdd', 'cmk', 'codA', 'crr', 'dapA', 'dapB', 'dapD', 'dapE', 'dapF', 
    'dcd', 'deoA', 'deoB', 'deoD', 'dmlA', 'dut', 'eno', 'fbaA', 'fbaB', 'fbp', 
    'fumA', 'fumB', 'fumC', 'fumD', 'fumE', 'gapA', 'glpX', 'gltA', 'glyA', 'gmk', 
    'gnd', 'gpmA', 'gpmM', 'gpt', 'gsk', 'guaA', 'guaB', 'guaD', 'hisA', 'hisB', 
    'hisC', 'hisD', 'hisF', 'hisG', 'hisH', 'hisI', 'hpt', 'iaaA', 'icd', 'ilvA', 
    'ilvB', 'ilvC', 'ilvD', 'ilvE', 'ilvH', 'ilvI', 'ilvM', 'ilvN', 'leuA', 'leuB', 
    'leuC', 'leuD', 'lpd', 'lysA', 'lysC', 'maeA', 'maeB', 'malY', 'mazG', 'mdh', 
    'metA', 'metB', 'metC', 'metE', 'metH', 'metL', 'mqo', 'nadD', 'nadE', 'nadR', 
    'ndk', 'nrdD', 'nudI', 'nupC', 'nupG', 'pck', 'pfkA', 'pfkB', 'pgi', 'pgk', 
    'pheA', 'phoA', 'pncA', 'pncB', 'ppnN', 'ppnP', 'ppsA', 'prs', 'ptsG', 'purA', 
    'purB', 'purC', 'purD', 'purE', 'purF', 'purH', 'purK', 'purL', 'purM', 'purN', 
    'purT', 'pykA', 'pykF', 'pyrC', 'pyrD', 'pyrE', 'pyrF', 'pyrG', 'pyrH', 'rihA', 
    'rihB', 'rihC', 'rpe', 'rpiA', 'rpiB', 'rutA', 'rutB', 'rutD', 'rutE', 'rutG', 
    'serA', 'serB', 'serC', 'sucA', 'sucB', 'sucC', 'sucD', 'talA', 'talB', 'tdk', 
    'thrA', 'thrB', 'thrC', 'thyA', 'tktA', 'tktB', 'tmk', 'tpiA', 'trpA', 'trpB', 
    'trpC', 'trpD', 'trpE', 'tyrA', 'tyrB', 'udk', 'udp', 'umpG', 'umpH', 'upp', 
    'ushA', 'xapA', 'xdhA', 'xdhB', 'xdhC', 'ybhA', 'ydfG', 'yggF', 'yrfG', 'zwf']

In [5]:
%time precalculated = utils.returnCommonNames('ECOLI')

CPU times: user 2.57 s, sys: 2.28 s, total: 4.85 s
Wall time: 50.6 s


In [6]:
%time network = utils.metabolicNetwork.FromGeneList('ECOLI', genes, precalculated = precalculated, fmt = 'genes')
network

CPU times: user 2.6 s, sys: 2.1 s, total: 4.7 s
Wall time: 42.7 s


,GENE OR COMPLEX,ENZYME LOCATION,REACTION,SUBSTRATES,PRODUCTS,FWD_RATE,RVS_RATE
0,aceA,unknown,ISOCIT-CLEAV-RXN,THREO-DS-ISO-CITRATE,"GLYOX,SUC",1.0,1.0
1,aceB,cytosol,MALSYN-RXN,"ACETYL-COA,WATER,GLYOX","PROTON,MAL,CO-A",1.0,0.0
2,aceE,cytosol,RXN0-1134,"PYRUVATE,Pyruvate-dehydrogenase-lipoate,PROTON","Pyruvate-dehydrogenase-acetylDHlipoyl,CARBON-D...",1.0,0.0
3,aceE,cytosol,PYRUVDEH-RXN,"PYRUVATE,CO-A,NAD","ACETYL-COA,CARBON-DIOXIDE,NADH",1.0,0.0
4,aceF,unknown,RXN0-1133,"ACETYL-COA,Pyruvate-dehydrogenase-dihydrolipoate","CO-A,Pyruvate-dehydrogenase-acetylDHlipoyl",0.0,1.0
...,...,...,...,...,...,...,...
453,ydfG,cytosol,RXN-8974,"3-HYDROXY-PROPIONATE,NADP","MALONATE-S-ALD,NADPH,PROTON",0.0,1.0
454,yggF,cytosol,F16BDEPHOS-RXN,"FRUCTOSE-16-DIPHOSPHATE,WATER","FRUCTOSE-6P,Pi",1.0,0.0
455,yrfG,cytosol,RXN-7607,"IMP,WATER","INOSINE,Pi",1.0,0.0
456,yrfG,cytosol,RXN-7609,"GMP,WATER","GUANOSINE,Pi",1.0,0.0


In [7]:
%time model = atlas.construct_model_from_metabolic_network(network, verbose = False)
model

It was found duplicated reaction names in the network.
Please check the conflicting_reactions.txt and correct them if necessary.
CPU times: user 51.6 s, sys: 0 ns, total: 51.6 s
Wall time: 51.7 s


<Model 'atlas_rbm.construct_model_from_metabolic_network' (monomers: 2, rules: 330, parameters: 1177, expressions: 0, compartments: 0) at 0x7f7422baf430>

<a id='figS1'></a><img src="central-carbon-metabolism.png" alt="drawing" width="1000"/>